# 1. Prepare data

Test the writing and reading performance on GeoLake.


We are going to use a Portaxi dataset which has 2m records. You can find it here: https://star.cs.ucr.edu/?portotaxi#center=41.1636,-8.5872&zoom=13

In [ ]:
%%shell

wget --no-verbose https://star.cs.ucr.edu/datasets/portotaxi/download.geojson.gz -O - | gzip -d > /home/iceberg/data/portotaxi.geojson
chmod 777 /home/iceberg/data/portotaxi.geojson

# this takes a while, file has about 3,5GB and trasnfer speeds are pretty low

We can also validate if the portotaxi file was correctly downloaded by run the below code

In [ ]:
%%shell

[ "$(md5sum /home/iceberg/data/portotaxi.geojson | cut -d ' ' -f1)" = "bd11ae6f439da60c9a2768c6f87af5bc" ] && echo "MD5 hash matches expected value." || echo "MD5 hash does not match expected value."

We need to convert the file from multiline to line delimited so we can read it with Spark.

In [ ]:
%%python
import ijson
import jsonlines

num_rows_missing_geometry = 0
input_path = "/home/iceberg/data/portotaxi.geojson"
output_path = "/home/iceberg/data/portotaxi.geojsonl"

with open(input_path) as f, jsonlines.open(output_path, mode="w") as writer:
    for feature in ijson.items(f, "features.item", use_float=True):
        if feature.get("geometry"):
            writer.write(feature)
        else:
            num_rows_missing_geometry += 1
print(f"Finished writing file: {output_path}. {num_rows_missing_geometry} rows were missing geometry and were skipped.")


In [ ]:
%%shell
chmod 777 /home/iceberg/data/portotaxi.geojsonl
# make the files accessible from outside docker

In [ ]:
%%init_spark

# You can adjust memory and other spark settings here

launcher.num_executors = 1
launcher.executor_cores = 8
launcher.driver_memory = '16g'
launcher.executor_memory = '16g'
launcher.conf.set("spark.driver.maxResultSize","4g")

In [ ]:
%%shell

# if no enough memory, you can split the file and only read the first 10k records
# uncomment line below to prepare smaller file
#pip install geojsplit && cd /home/iceberg/data/ && geojsplit -n 1 --geometry-count 10000 portotaxi.geojson && ls -lh /home/iceberg/data/

Create DataFrame out of line-delimited file

In [ ]:
import org.apache.spark.sql.types._

val schema = new StructType()
    .add("geometry", StringType)
    .add("properties", new StructType()
        .add("CALL_TYPE", StringType)
        .add("DAY_TYPE", StringType)
        .add("MISSING_DATA", BooleanType)
        .add("ORIGIN_CALL", StringType)
        .add("ORIGIN_STAND", StringType)
        .add("TAXI_ID", LongType)
        .add("TIMESTAMP", StringType)
        .add("TRIP_ID", LongType)
    )
    .add("type", StringType)

val df = spark
    .read
    .schema(schema)
    .json("/home/iceberg/data/portotaxi.geojsonl")
    .selectExpr("properties.*", "ST_GeomFromGeoJSON(geometry) as geometry")

df.show()

Create temp view `portotaxi` with columns that we need

In [ ]:
df
.select(
    "TRIP_ID",
    "CALL_TYPE",
    "ORIGIN_STAND",
    "TAXI_ID",
    "TIMESTAMP",
    "DAY_TYPE",
    "MISSING_DATA",
    "geometry"
)
.repartition(10)
.cache
.createOrReplaceTempView("portotaxi")

In [ ]:
spark.sql("SELECT * FROM portotaxi").count()

# 2. Benchmark of Parquet Encodings

## Create Tables

Create table with different geo-encodings.

In [ ]:
val geoEncodings = Seq("nested-list", "wkb-bbox", "wkb")
val tables = Seq("portotaxi_nested_list", "portotaxi_wkb_bbox", "portotaxi_wkb")

tables.zip(geoEncodings).foreach(x => {
    val sql = s"""CREATE TABLE IF NOT EXISTS demo.db.${x._1}
    (
      TRIP_ID LONG,
      CALL_TYPE STRING,
      ORIGIN_STAND STRING,
      TAXI_ID INTEGER,
      TIMESTAMP STRING,
      DAY_TYPE STRING,
      MISSING_DATA BOOLEAN,
      geometry GEOMETRY
    )
    USING iceberg
    TBLPROPERTIES ('write.parquet.geometry.encoding' = '${x._2}')
    """
    println(sql)
    spark.sql(sql)
})

## Writing



In [ ]:
tables.foreach(tb => {
    val t0 = System.currentTimeMillis()
    spark.sql(s"INSERT INTO demo.db.${tb} SELECT * FROM portotaxi")
    val t1 = System.currentTimeMillis()
    println(s"time cost on table ${tb}: ${(t1 - t0) / 1000.0}s")
})

In [ ]:
val sql = tables.map(t => s"""
(SELECT '${t}' as table, 
    summary['total-records'] as total_records,
    round(summary['total-files-size'] / 1024 / 1024, 2) as file_size_in_mb
 FROM demo.db.${t}.snapshots)
""").reduce(_ + " UNION " + _)
spark.sql(sql).show()

## Reading

In [ ]:
val bbox = "POLYGON ((-8.6079 41.1489, -8.6089 41.1472, -8.6066 41.1470, -8.6061 41.1483, -8.6079 41.1489))"
tables.foreach(t => {
    val t0 = System.currentTimeMillis()
    spark.sql(s"SELECT count(*) FROM demo.db.${t} WHERE ST_Within(geometry, IcebergSTGeomFromText('${bbox}'))").show()
    val t1 = System.currentTimeMillis()
    println(s"time cost on table ${t}: ${(t1 - t0) / 1000.0}s")
})

# 3. Benchmark of Partitions


Create tables with different partition resolution: 3, 7, 11, 15, 19.

In [ ]:
val resolutions = Seq(3, 7, 11, 15, 19)
resolutions.foreach(r => {
    val sql = s"""
    CREATE TABLE IF NOT EXISTS demo.db.portotaxi_xz${r}
    (
      TRIP_ID LONG,
      CALL_TYPE STRING,
      ORIGIN_STAND STRING,
      TAXI_ID INTEGER,
      TIMESTAMP STRING,
      DAY_TYPE STRING,
      MISSING_DATA BOOLEAN,
      geometry GEOMETRY
    )
    USING iceberg
    PARTITIONED BY (xz2(geometry, ${r}))
    TBLPROPERTIES ('write.parquet.geometry.encoding' = 'nested-list')    
    """
    spark.sql(sql)
    val t0 = System.currentTimeMillis()
    spark.sql(s"INSERT INTO demo.db.portotaxi_xz${r} SELECT * FROM portotaxi")
    val t1 = System.currentTimeMillis()
    println(s"time cost on table demo.db.portotaxi_xz${r}: ${(t1 - t0) / 1000.0}s")
})


Number of partitions and data files in each table:

In [ ]:
val sql = resolutions.map(t => s"(SELECT ${t} as resolution, summary['changed-partition-count'] as partitions,summary['total-data-files'] as total_data_files FROM demo.db.portotaxi_xz${t}.snapshots)").reduce(_ + " UNION " + _) + " ORDER BY resolution"

spark.sql(sql).show()

Reading speed:

In [ ]:
resolutions.foreach(t => {
    val t0 = System.currentTimeMillis()
    spark.sql(s"SELECT count(*) FROM demo.db.portotaxi_xz${t} WHERE ST_Within(geometry, ST_GeomFromText('${bbox}'))").show()
    val t1 = System.currentTimeMillis()
    println(s"time cost on resolutions ${t}: ${(t1 - t0) / 1000.0}s")
})